### Model_loading
---

In [1]:
from tensorflow.keras.models import load_model
import h5py
import pandas as pd
import numpy as np
import io
from PIL import Image

In [2]:
DenseNet_model = load_model("/kaggle/input/skincancer_classification/keras/default/1/model_densenet.keras")
ResNet_model = load_model("/kaggle/input/skincancer_classification/keras/default/1/model_resnet.keras")
Xception_model = load_model("/kaggle/input/skincancer_classification/keras/default/1/model_xception.keras")

In [3]:
dataset = h5py.File("/kaggle/input/isic-2024-challenge/test-image.hdf5","r")

In [4]:
def preprocess_image(image, crop_width, crop_height):
    img_plt = Image.open(io.BytesIO(np.array(image)))

    width, height = img_plt.size
    left = (width - crop_width) / 2
    top = (height - crop_height) / 2
    right = (width + crop_width) / 2
    bottom = (height + crop_height) / 2
    img_cropped = img_plt.crop((left, top, right, bottom))
 
    return img_cropped


def data_gen_test(test_set, batch_size, crop_width=90, crop_height=90):
    keys = list(test_set.keys())
    for i in range(0, len(keys), batch_size):
        batch_keys = keys[i:i + batch_size]
        test_data = []
        for j in batch_keys:
            img_array = preprocess_image(test_set[j], crop_width, crop_height)
            test_data.append(img_array)
            print(j)
        yield np.array(test_data)


In [5]:
predictions_a = []
predictions_b = []
predictions_c = []

for test_batch in data_gen_test(dataset, batch_size=10000):
    
    print(test_batch.shape)   
    batch_predictions_1 = DenseNet_model.predict(test_batch)
    batch_predictions_2 = ResNet_model.predict(test_batch)
    batch_predictions_3 = Xception_model.predict(test_batch)
    print(batch_predictions_1.shape)
    
    predictions_a.extend(batch_predictions_1.ravel())
    predictions_b.extend(batch_predictions_2.ravel())
    predictions_c.extend(batch_predictions_3.ravel())
    

ISIC_0015657
ISIC_0015729
ISIC_0015740
(3, 90, 90, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
(3, 1)


In [6]:
x = np.array(predictions_a)
y = np.array(predictions_b)
z = np.array(predictions_c)

In [7]:
final_pred = (x+y+z)/3


In [8]:
submission = pd.DataFrame({  
    "isic_id": list(dataset.keys()),  
    "target": final_pred
})  

# Save the DataFrame to a CSV file without scientific notation  
submission.to_csv("/kaggle/working/submission.csv", index=False, float_format='%.7f') 